In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns



import numpy as np
import pandas as pd

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['figure.figsize'] = np.array((10,6))*.6

# Load galaxy image data

In [2]:
X_img = np.load("data/images.small.npy")
X_img = X_img.transpose([0,2,3,1])
X_img.shape

(1866, 50, 50, 3)

In [3]:
image_size = X_img.shape[1]
image_size

50

# Load targets
`HSC_ids` are in the same order as the `X_img` data. These ids are then used to cross-reference the table read into `df`.

In [4]:
HSC_ids = np.load("data/HSC_ids.npy")
HSC_ids

array([43158176442374224, 43158176442374373, 43158176442374445, ...,
       43159155694916013, 43159155694916476, 43159155694917496])

In [5]:
df = pd.read_csv("data/2018_02_23-all_objects.csv")
df = df[df.selected]


df = df.drop_duplicates("HSC_id") \
       .set_index("HSC_id") \
       [["photo_z", "log_mass"]]


df.head()


,photo_z,log_mass
HSC_id,,
43158584464268619,0.4810,9.17510
43158721903220850,0.0050,7.64999
43158447025313043,0.0857,7.94762
43158584464268728,0.1315,8.21643
43158447025292832,0.0701,7.86620


In [6]:
y = df.loc[HSC_ids].values

# Run GAN
Modeled after: https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/f24a27feba327a1086298a810fdf83bb30d5128a/CGAN.py

In [7]:
import tensorflow as tf

import gan

In [8]:
sess = tf.Session()
num_epochs = 170
batch_size = 64
z_dim = 100
dataset_name = "galaxy"
checkpoint_dir = "models/gan/checkpoints"
result_dir = "models/gan/results"
log_dir = "models/gan/log"

model = gan.CGAN(sess, num_epochs, batch_size, z_dim, dataset_name,
                 image_size, X_img, df.loc[HSC_ids].values,
                 checkpoint_dir, result_dir, log_dir, )

model.build_model()


In [10]:
model.train()

 [*] Reading checkpoints...
 [*] Success to read CGAN.model-4930
 [*] Load SUCCESS
